In [1]:
from langchain.document_loaders.csv_loader import CSVLoader

loader = CSVLoader(file_path="./RAG_files/Test_demo.csv")
data = loader.load()

In [2]:
import pandas as pd

ex = pd.read_csv('./RAG_files/Test_demo.csv')
ex

,Submitter's Name,ID,Connected Name,Address,Telephone,Email,Withhold,Oppose?,Support,Heared,Not wish Heared,Joint,No Joint,Share detail,No Share detial,Te Reo?,Maori Commissioner?,Unnamed: 17
0,Ray,1,Ray Wang,121 Solomon Street,021 0849 6271,kyrie19960324@gmail.com,True,True,False,False,True,False,True,False,True,False,True,NaN
1,Dawn Keller,2,Dawn Keller,"161 Clarke Road, Te Puna, Tauranga",NaN,NaN,True,True,False,False,True,True,False,False,True,False,True,NaN
2,"Denis Hirano,",3,Denis Hilanna,NaN,NaN,NaN,True,True,True,False,True,True,False,False,True,True,True,NaN
3,HUGH PATRICK CLARK.,4,NaN,"41 CLARKE ROAD, TE PUNA R. D.G.",2108788874,hughclark@hotmail.com.,False,True,False,False,True,True,False,True,False,False,True,NaN
4,Lulu Monette Johnson,5,Monette,159C Te Puna Road,212861179,wenvoe2@gmail.com,False,True,False,False,True,NaN,True,False,False,False,True,NaN
5,COLIN HERBERT HYDE,6,COLIN HERBERT HYDE,138 TEPAINA ROADR. B.A 3174,220352207,PAM@ TOUCHNZ.CO.NZ.,True,True,False,False,True,True,False,True,False,False,True,NaN
6,COLIN HERBERT HYDE,7,COLIN HERBERT HYDE,138 TEPAINA ROADR. B.A 3174,220352207,PAM@ TOUCHNZ.CO.NZ.,True,True,False,False,True,True,False,True,False,False,True,NaN


In [3]:
from langchain.embeddings import LlamaCppEmbeddings
from langchain.vectorstores import Chroma

n_gpu_layers = 1  # Metal set to 1 is enough.
n_batch = 512  # Should be between 1 and n_ctx, consider the amount of RAM of your Apple Silicon Chip.
use_mlock = True  # Force system to keep model in RAM.

# Make sure the model path is correct for your system!
model_path="/Users/zwang/Desktop/pgpt/privateGPT/models/cache/models--TheBloke--Mistral-7B-Instruct-v0.2-GGUF/blobs/3e0039fd0273fcbebb49228943b17831aadd55cbcbf56f0af00499be2040ccf9"

embedding = LlamaCppEmbeddings(
    model_path=model_path,
    n_gpu_layers=n_gpu_layers,
    n_batch=n_batch,
    n_ctx=2048,
    f16_kv=True,  # MUST set to True, otherwise you will run into problem after a couple of calls
    use_mlock = True
)

llama_model_loader: loaded meta data with 24 key-value pairs and 291 tensors from /Users/zwang/Desktop/pgpt/privateGPT/models/cache/models--TheBloke--Mistral-7B-Instruct-v0.2-GGUF/blobs/3e0039fd0273fcbebb49228943b17831aadd55cbcbf56f0af00499be2040ccf9 (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = mistralai_mistral-7b-instruct-v0.2
llama_model_loader: - kv   2:                       llama.context_length u32              = 32768
llama_model_loader: - kv   3:                     llama.embedding_length u32              = 4096
llama_model_loader: - kv   4:                          llama.block_count u32              = 32
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 14336
llama_model

KeyboardInterrupt: 

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=0)
all_splits = text_splitter.split_documents(data)


In [ ]:
vectorstore = Chroma.from_documents(documents=all_splits, embedding=embedding)


llama_print_timings:        load time =    1637.95 ms
llama_print_timings:      sample time =       0.00 ms /     1 runs   (    0.00 ms per token,      inf tokens per second)
llama_print_timings: prompt eval time =    1637.06 ms /   132 tokens (   12.40 ms per token,    80.63 tokens per second)
llama_print_timings:        eval time =       0.00 ms /     1 runs   (    0.00 ms per token,      inf tokens per second)
llama_print_timings:       total time =    1641.59 ms

llama_print_timings:        load time =    1637.95 ms
llama_print_timings:      sample time =       0.00 ms /     1 runs   (    0.00 ms per token,      inf tokens per second)
llama_print_timings: prompt eval time =    1149.44 ms /   118 tokens (    9.74 ms per token,   102.66 tokens per second)
llama_print_timings:        eval time =       0.00 ms /     1 runs   (    0.00 ms per token,      inf tokens per second)
llama_print_timings:       total time =    1154.40 ms

llama_print_timings:        load time =    1637.95 ms
l

In [ ]:
from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain.llms import LlamaCpp

callback_manager = CallbackManager([StreamingStdOutCallbackHandler()])

# Make sure the model path is correct for your system!
llm = LlamaCpp(
    model_path=model_path,
    n_gpu_layers=n_gpu_layers,
    n_batch=n_batch,
    n_ctx=2048,
    #f16_kv=True,  # MUST set to True, otherwise you will run into problem after a couple of calls
    callback_manager=callback_manager,
    verbose=True,
    use_mlock = True
)

user_question = "What does ID 1 submit looks like? List all the detail like: Is he Oppose or support? And what is he Telephone number?"

from langchain.chains import RetrievalQA

from langchain.prompts import PromptTemplate

prompt_template = """Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

{context}

Question: {question}
Helpful Answer:"""
PROMPT = PromptTemplate(
    template=prompt_template, input_variables=["context", "question"]
)

qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever=vectorstore.as_retriever(),
    chain_type_kwargs={"prompt": PROMPT},
)

answer = qa_chain({"query": user_question})
print(answer)

llama_model_loader: loaded meta data with 24 key-value pairs and 291 tensors from /Users/zwang/Desktop/pgpt/privateGPT/models/cache/models--TheBloke--Mistral-7B-Instruct-v0.2-GGUF/blobs/3e0039fd0273fcbebb49228943b17831aadd55cbcbf56f0af00499be2040ccf9 (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = mistralai_mistral-7b-instruct-v0.2
llama_model_loader: - kv   2:                       llama.context_length u32              = 32768
llama_model_loader: - kv   3:                     llama.embedding_length u32              = 4096
llama_model_loader: - kv   4:                          llama.block_count u32              = 32
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 14336
llama_model

 The information provided for ID 1 submitter includes their Connected Name (Denis Hilanna), Address, Telephone Number, Email, Withhold status, Oppose status, and Support status. Based on the information given, Denis Hilanna is listed as Opposing. His telephone number is not explicitly stated in the provided context, but it can be found within his address details: 41 CLARKE ROAD, TE PUNA R. D.G.{'query': 'What does ID 1 submit looks like? List all the detail like: Is he Oppose or support? And what is he Telephone number?', 'result': ' The information provided for ID 1 submitter includes their Connected Name (Denis Hilanna), Address, Telephone Number, Email, Withhold status, Oppose status, and Support status. Based on the information given, Denis Hilanna is listed as Opposing. His telephone number is not explicitly stated in the provided context, but it can be found within his address details: 41 CLARKE ROAD, TE PUNA R. D.G.'}



llama_print_timings:        load time =    5107.28 ms
llama_print_timings:      sample time =      27.36 ms /    98 runs   (    0.28 ms per token,  3582.00 tokens per second)
llama_print_timings: prompt eval time =    6006.12 ms /   604 tokens (    9.94 ms per token,   100.56 tokens per second)
llama_print_timings:        eval time =    8758.23 ms /    97 runs   (   90.29 ms per token,    11.08 tokens per second)
llama_print_timings:       total time =   15189.86 ms


In [ ]:
answer

{'query': 'What does ID 1 submit looks like? List all the detail like: Is he Oppose or support? And what is he Telephone number?',
 'result': ' The information provided for ID 1 submitter includes their Connected Name (Denis Hilanna), Address, Telephone Number, Email, Withhold status, Oppose status, and Support status. Based on the information given, Denis Hilanna is listed as Opposing. His telephone number is not explicitly stated in the provided context, but it can be found within his address details: 41 CLARKE ROAD, TE PUNA R. D.G.'}

In [ ]:
user_question2 = "What does ID 2 submit looks like? List all the detail like: Is he Oppose or support? And what is he Telephone number?"
answer = qa_chain({"query": user_question2})
print(answer)


llama_print_timings:        load time =    1637.95 ms
llama_print_timings:      sample time =       0.00 ms /     1 runs   (    0.00 ms per token,      inf tokens per second)
llama_print_timings: prompt eval time =     510.46 ms /    31 tokens (   16.47 ms per token,    60.73 tokens per second)
llama_print_timings:        eval time =       0.00 ms /     1 runs   (    0.00 ms per token,      inf tokens per second)
llama_print_timings:       total time =     514.45 ms
Llama.generate: prefix-match hit


 The submitter with ID 2 is named Dawn Keller. Her opposition status is indicated as 'TRUE'. Her telephone number is missing from the provided details. The rest of her contact information includes her connected name, address, email and withhold status.{'query': 'What does ID 2 submit looks like? List all the detail like: Is he Oppose or support? And what is he Telephone number?', 'result': " The submitter with ID 2 is named Dawn Keller. Her opposition status is indicated as 'TRUE'. Her telephone number is missing from the provided details. The rest of her contact information includes her connected name, address, email and withhold status."}



llama_print_timings:        load time =    5107.28 ms
llama_print_timings:      sample time =      12.92 ms /    53 runs   (    0.24 ms per token,  4102.17 tokens per second)
llama_print_timings: prompt eval time =    3071.33 ms /   304 tokens (   10.10 ms per token,    98.98 tokens per second)
llama_print_timings:        eval time =    4702.65 ms /    52 runs   (   90.44 ms per token,    11.06 tokens per second)
llama_print_timings:       total time =    7990.67 ms


In [ ]:
answer['result']

" The submitter with ID 2 is named Dawn Keller. Her opposition status is indicated as 'TRUE'. Her telephone number is missing from the provided details. The rest of her contact information includes her connected name, address, email and withhold status."